# Reconstruction with Multi-Shell Multi-Tissue CSD

This tutorial is based off the example provided here: https://github.com/dipy/dipy/blob/master/doc/examples/reconst_mcsd.py

This example shows how to fit Multi-Shell Multi-Tissue Constrained Spherical
Deconvolution (MSMT-CSD) using Ray for parallelization. 

In [ ]:
from paths import afq_home

In [ ]:
import matplotlib.pyplot as plt
import os.path as op
import numpy as np

import AFQ.data.fetch as afd

from dipy.core.gradients import gradient_table, unique_bvals_tolerance
from dipy.data import get_sphere
import dipy.direction.peaks as dp
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti
from dipy.reconst.mcsd import (
    MultiShellDeconvModel,
    mask_for_response_msmt,
    multi_shell_fiber_response,
    response_from_mask_msmt,
)
import dipy.reconst.shm as shm
from dipy.segment.tissue import TissueClassifierHMRF
from dipy.segment.mask import median_otsu

## Download dataset

We use the fetch function to download a multi-shell dataset provided by Hansen and Jespersen.


In [ ]:
sphere = get_sphere(name="symmetric724")
study_dir = afd.fetch_hbn_preproc(["NDARAA948VFH"])[1]
sub_dir = op.join(study_dir, "derivatives/qsiprep/sub-NDARAA948VFH")

fraw = op.join(sub_dir, "ses-HBNsiteRU/dwi/sub-NDARAA948VFH_ses-HBNsiteRU_acq-64dir_space-T1w_desc-preproc_dwi.nii.gz")
fbval = op.join(sub_dir, "ses-HBNsiteRU/dwi/sub-NDARAA948VFH_ses-HBNsiteRU_acq-64dir_space-T1w_desc-preproc_dwi.bval")
fbvec = op.join(sub_dir, "ses-HBNsiteRU/dwi/sub-NDARAA948VFH_ses-HBNsiteRU_acq-64dir_space-T1w_desc-preproc_dwi.bvec")
t1_fname = op.join(sub_dir, "anat/sub-NDARAA948VFH_desc-preproc_T1w.nii.gz")

data, affine = load_nifti(fraw)
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab = gradient_table(bvals, bvecs=bvecs)

## Generate brain mask

We make use of the `median_otsu` method to generate the mask.


In [ ]:
b0_mask, mask = median_otsu(data, median_radius=2, numpass=1, vol_idx=[0, 1])


## Generate Anisotropic Power Map


In [ ]:
qball_model = shm.QballModel(gtab, 8)
peaks = dp.peaks_from_model(
    model=qball_model,
    data=data,
    relative_peak_threshold=0.5,
    min_separation_angle=25,
    sphere=sphere,
    mask=mask,
)
ap = shm.anisotropic_power(peaks.shm_coeff)
plt.matshow(np.rot90(ap[:, :, 50]), cmap=plt.cm.bone)
plt.show()

In [ ]:
plt.close()

## Tissue Classification using HMRF

We use Hidden Markov Random Fields for tissue classification.


In [ ]:
nclass = 3
beta = 0.1
hmrf = TissueClassifierHMRF()
initial_segmentation, final_segmentation, PVE = hmrf.classify(ap, nclass, beta)

csf = np.where(final_segmentation == 1, 1, 0)
gm = np.where(final_segmentation == 2, 1, 0)
wm = np.where(final_segmentation == 3, 1, 0)

## Estimate response functions


In [ ]:
mask_wm, mask_gm, mask_csf = mask_for_response_msmt(
    gtab,
    data,
    roi_radii=10,
    wm_fa_thr=0.7,
    gm_fa_thr=0.3,
    csf_fa_thr=0.15,
    gm_md_thr=0.001,
    csf_md_thr=0.0032,
)

mask_wm *= wm
mask_gm *= gm
mask_csf *= csf

response_wm, response_gm, response_csf = response_from_mask_msmt(
    gtab, data, mask_wm, mask_gm, mask_csf
)

print(response_wm)
print(response_gm)
print(response_csf)

## Reconstruction with MSMT-CSD


In [ ]:
ubvals = unique_bvals_tolerance(gtab.bvals)
response_mcsd = multi_shell_fiber_response(
    sh_order_max=8,
    bvals=ubvals,
    wm_rf=response_wm,
    gm_rf=response_gm,
    csf_rf=response_csf,
)

mcsd_model = MultiShellDeconvModel(gtab, response_mcsd)
mcsd_fit = mcsd_model.fit(data[:, :, 50], engine="ray") # Using a subset of the data for speed in this example

In [ ]:
ap = shm.anisotropic_power(mcsd_fit.shm_coeff)
plt.matshow(np.rot90(ap[:, :]), cmap=plt.cm.bone)
plt.show()

In [ ]:
plt.close()

## References

- Jeurissen, B., et al. "Multi-tissue constrained spherical deconvolution for improved analysis of multi-shell diffusion MRI data." NeuroImage 2014.
- Tournier, J-D., et al. "Robust determination of the fibre orientation distribution in diffusion MRI." NeuroImage 2007.
